<a href="https://colab.research.google.com/github/Medynal/thing/blob/main/PRAC_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Brownfield Redevelopment Tax Credits are incentives given to encourage the clean-up and redevelopment of brownfields i.e. properties that are abandoned or underutilized due to contamination or perceived contamination.
The Department of Taxation and Finance annually produces a mandated report of taxpayers claiming Brownfield Redevelopment credit to offset their tax. The aim of this analysis is to explore the policy impact in New York City (does the program improve the number of brownfields developed annually in New York City?)


In [1]:
!pip -q install gdown==4.6.0
import gdown
gdrive_link = 'https://drive.google.com/file/d/1zcUPAntkZVHe3rtbC0sOX_Etv56Kdx4E/view?usp=sharing'
file_id= '1zcUPAntkZVHe3rtbC0sOX_Etv56Kdx4E'
downloadable_link= f'https://drive.google.com/uc?id={file_id}'
gdown.download(downloadable_link, output= 'NYC_brownfields.csv')
print ('download complete!' )

Downloading...
From: https://drive.google.com/uc?id=1zcUPAntkZVHe3rtbC0sOX_Etv56Kdx4E
To: /content/NYC_brownfields.csv
100%|██████████| 237k/237k [00:00<00:00, 65.2MB/s]

download complete!


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

brownfield_df= pd.read_csv('NYC_brownfields.csv')
print(f'shape: {brownfield_df.shape}\n')
print(f'Missin Values:\n {brownfield_df.isna().sum()}')
print('\nData Information:')
brownfield_df.info()

shape: (1461, 19)

Missin Values:
 Calendar Year                                0
Tax Year                                     0
Taxpayer Name                                0
Project Site Code                            0
Project Site Name                            0
Notes                                      389
DEC Region                                   0
County                                       0
Municipality                                 0
En-Zone                                      0
 Site Preparation Component Costs            1
Site Preparation Component Credit            1
 Tangible Property Component Costs           3
Tangible Property Component Credit           1
 OnSite Groundwater Remediation Cost         3
 OnSite Groundwater Remediation Credit       5
 Total Cost                                  1
 Total Credit                                0
Georeference                                 3
dtype: int64

Data Information:
<class 'pandas.core.frame.DataFrame'>
Ra

below are the description of each column
1. Calendar Year:Year when the project or credit was recorded.
2.	Tax Year: Fiscal year for which the tax credit applies.
3.	Taxpayer Name: Name of the company receiving the credit.
4.	Project Site Code: Unique ID for each brownfield site.
5.	Project Site Name:Name or of the brownfield site.
6.	Notes:Remarks about the project or credit.
7.	DEC Region:The New York State Department of Environmental Conservation region where the brownfield is located.
8.	County: County where the brownfield site is based.
9.	Municipality: Local municipality (e.g., city, town) overseeing the site.
10.	En-Zone: Indicates if the site is within an “Environmental Zone” (areas targeted for economic redevelopment).
11.	Site Preparation Component Costs: Total cost of preparing the site (cleanup, demolition, etc.).
12.	Site Preparation Component Credit: Tax credit amount awarded for site preparation costs
13.	Tangible Property Component Costs: Costs related to new tangible assets (buildings, infrastructure).
14.	Tangible Property Component Credit: Tax credit amount awarded for tangible property investments.
15.	OnSite Groundwater Remediation Cost: Costs spent specifically on cleaning contaminated groundwater.
16.	OnSite Groundwater Remediation Credit:Tax credit amount awarded for groundwater cleanup.
17.	Total Cost: Sum of all costs for the project (preparation + tangible + remediation).
18.	Total Credit: Total amount of tax credits granted for the project.
19.	Georeference: Geolocation data (coordinates or reference ID) for mapping purposes.


In [3]:
#rename columns with long names for easy access.
brownfield_df.columns = brownfield_df.columns.str.strip()
brownfield_df.rename(columns={'Site Preparation Component Costs':'spc_cost',
                              'Site Preparation Component Credit':'spc_credit',
                              'Tangible Property Component Costs':'tpc_cost',
                              'Tangible Property Component Credit':'tpc_credit',
                              'OnSite Groundwater Remediation Cost':'ogrc_cost',
                              'OnSite Groundwater Remediation Credit':'ogrc_credit'},inplace=True)
brownfield_df.columns= brownfield_df.columns.str.lower()



In [4]:
brownfield_df

,calendar year,tax year,taxpayer name,project site code,project site name,notes,dec region,county,municipality,en-zone,spc_cost,spc_credit,tpc_cost,tpc_credit,ogrc_cost,ogrc_credit,total cost,total credit,georeference
0,2022,2021,NEW 470 LLC,C224242,New 470 Project,"4/, 7/",2,Kings,Brooklyn,No,0.0,0.0,103861.0,15579.0,0.0,0.0,103861.0,15579,POINT (-73.99036 40.69245)
1,2009,2008,EMPIRE GEN HOLDINGS INC,C442035,South 40 Site,NaN,4,Rensselaer,Rensselaer,No,700000.0,84000.0,0.0,0.0,0.0,0.0,700000.0,84000,POINT (-73.74127 42.64472)
2,2020,2019,NEW 470 LLC,C224242,New 470 Project,"4/, 7/",2,Kings,Brooklyn,No,2935248.0,821869.0,0.0,0.0,0.0,0.0,2935248.0,821869,POINT (-73.99036 40.69245)
3,2018,2017,BLA II LLC,C915262,125 Main Street Site,3/,9,Erie,Buffalo,No,0.0,0.0,43537.0,4354.0,0.0,0.0,43537.0,4354,POINT (-78.87846 42.88545)
4,2016,2015,BLA II LLC,C915262,125 Main Street Site,3/,9,Erie,Buffalo,No,0.0,0.0,32274.0,3227.0,0.0,0.0,32274.0,3227,POINT (-78.87846 42.88545)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2018,2017,SENECA MARKET I LLC,C849004,Seneca Market I LLC,NaN,8,Schuyler,Watkins Glen,Yes,0.0,0.0,10768.0,1077.0,0.0,0.0,10768.0,1077,POINT (-76.87373 42.38107)
1457,2024,2023,MAIN & DODGE LLC,C915341,1155 Main Street,4/,9,Erie,Buffalo,Yes,0.0,0.0,67819.0,13564.0,0.0,0.0,67819.0,13564,POINT (-78.87846 42.88545)
1458,2017,2016,ZURENDA ENTERPRISES INC,C704045,TCMF Hillcrest Facility,NaN,7,Broome,Binghamton,No,0.0,0.0,0.0,0.0,68349.0,8202.0,68349.0,8202,POINT (-75.91129 42.09871)
1459,2009,2008,SENECA MARKET I LLC,C849004,Seneca Market I LLC,NaN,8,Schuyler,Watkins Glen,Yes,1368788.0,136878.0,16040015.0,1604002.0,0.0,0.0,17408803.0,1740880,POINT (-76.87373 42.38107)


Assumption: Most credit awarded are half of the cost accrued. therefore, it is assumed that:
Missing credit is half of the cost if available
Missing cost is twice the credit, if available
If both are missing, then both are zero.

For missing geospatial reference, it is assumed that project with the same project site code will have the same georeference code

In [5]:
#Data Cleaning
## dealing with mising values
def na_values(row, a, b):
    if pd.isna(row[a]) and pd.notna(row[b]):
        row[a] = row[b]*2
    elif pd.notna(row[a]) and pd.isna(row[b]):
        row[b] = row[a]/2
    elif pd.isna(row[a]) and pd.isna(row[b]):
        row[a], row[b] = 0, 0
    return row

brownfield_df = brownfield_df.apply(na_values, axis=1, args=('spc_cost', 'spc_credit'))
brownfield_df = brownfield_df.apply(na_values, axis=1, args=('tpc_cost', 'tpc_credit'))
brownfield_df = brownfield_df.apply(na_values, axis=1, args=('ogrc_cost', 'ogrc_credit'))

total_cost = brownfield_df['spc_cost'] + brownfield_df['tpc_cost'] + brownfield_df['ogrc_cost']
total_credit = brownfield_df['spc_credit'] + brownfield_df['tpc_credit'] + brownfield_df['ogrc_credit']
brownfield_df['total cost']= brownfield_df['total cost'].fillna(total_cost)
##recalculating total cost and total credit due to fixed missing values
brownfield_df['total cost']= total_cost
brownfield_df['total credit']= total_credit







Rows with missing georeference value has the same project site code (C622031). other rows with the same project site code has a georeference of POINT (-74.98433 43.0259)


In [6]:
## show rows with missing georefernce
brownfield_df[brownfield_df['georeference'].isna()]
##check for complete rows with the same project site code
brownfield_df[brownfield_df['project site code']== 'C622031']
brownfield_df['georeference']= brownfield_df['georeference'].fillna('POINT (-74.98433 43.0259)')


In [7]:
##recheck for missing values
brownfield_df.isna().sum()

,0
calendar year,0
tax year,0
taxpayer name,0
project site code,0
project site name,0
notes,389
dec region,0
county,0
municipality,0
en-zone,0


In [8]:
## split the georefernce to longitude and latitude
def coordinate(g):
  g= g.strip()
  pattern = r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)'
  match = re.search(pattern, g)
  if match:
    longitude = float(match.group(1))
    latitude = float(match.group(2))
    return pd.Series([longitude, latitude])
  else:
    return pd.Series([None, None])

brownfield_df[['longitude', 'latitude']] = brownfield_df['georeference'].apply(coordinate)


In [9]:
## checking for missing values===> No missing values!
brownfield_df[brownfield_df['longitude'].isna()]


,calendar year,tax year,taxpayer name,project site code,project site name,notes,dec region,county,municipality,en-zone,...,spc_credit,tpc_cost,tpc_credit,ogrc_cost,ogrc_credit,total cost,total credit,georeference,longitude,latitude


In [10]:
## Drop column notes
brownfield_dffinal= brownfield_df.drop(columns='notes')
## checking final data details
print(f'shape: {brownfield_dffinal.shape}\n')
print(f'Missin Values:\n {brownfield_dffinal.isna().sum()}')
print('\nData Information:')
brownfield_dffinal.info()



shape: (1461, 20)

Missin Values:
 calendar year        0
tax year             0
taxpayer name        0
project site code    0
project site name    0
dec region           0
county               0
municipality         0
en-zone              0
spc_cost             0
spc_credit           0
tpc_cost             0
tpc_credit           0
ogrc_cost            0
ogrc_credit          0
total cost           0
total credit         0
georeference         0
longitude            0
latitude             0
dtype: int64

Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   calendar year      1461 non-null   int64  
 1   tax year           1461 non-null   int64  
 2   taxpayer name      1461 non-null   object 
 3   project site code  1461 non-null   object 
 4   project site name  1461 non-null   object 
 5   dec region         1461 n

In [11]:
##Save cleaned data
brownfield_dffinal.to_csv('NYC_brownfields_cleaned.csv', index=False)

Descriptive analysis of the dataset.

In [14]:
## print number of unique values in each column
n= brownfield_dffinal.nunique()
print('Number of unique values in each column: \n')
print(n)
brownfield_dffinal[['calender year', 'project site code']].groupby(['project site code']).nunique()

Number of unique values in each column: 

calendar year          18
tax year               18
taxpayer name         605
project site code     530
project site name     588
dec region              9
county                 41
municipality          119
en-zone                 2
spc_cost              745
spc_credit            745
tpc_cost              936
tpc_credit            933
ogrc_cost             193
ogrc_credit           193
total cost           1454
total credit         1451
georeference          107
longitude             107
latitude              107
dtype: int64


KeyError: "['carlender year'] not in index"